# Spatial Disorder Framework Manipulation
This notebook will walk you through the basic functionality of SDFM. Afterwards, you will know how to analyse and deconstruct frameworks into discrete building blocks, manipulate topological components and make reasonable molecular fragments.

**Disclaimer** This package is not thoroughly tested. Feel free to play around and discover bugs. It just works (sometimes) - Todd Howard. 

In [1]:
# some imports
from pathlib import Path

import ase
import ase.io
import ase.visualize

from sdfm.template import AtomsTemplate, graph_from_atoms
from sdfm.modify import ExtendedAtomsTemplate, StandaloneBuildingBlock
from sdfm.selection import AtomSelection, SelectByElement, SelectByChance, SelectByAdjacency, SelectByConnectivity, SelectByIndex, ReduceToComponent
from sdfm.gui import visualise_template_blocks, visualise_selection

root = Path.cwd()

Let's start by reading and inspecting a basic MOF structure: the prototypical UiO-66(Zr) framework. We will use the conventional 456-atom unit cell, containing 4 bricks and 24 linkers. 

In [2]:
file = root / 'data' / 'uio66_unit.xyz'
atoms = ase.io.read(file)
ase.visualize.view(atoms)

<Popen: returncode: None args: ['/home/pieter/micromamba/envs/psiflow_dev/bi...>

## The `AtomsTemplate` class

This package is built around the `AtomsTemplate` class, which wraps an `ase.Atoms` object and defines most functionality. It will create a topological network of atomic bonds and 'guess' a building block structure for the given system. To get the topology correct, it is important to start from a reasonable, relaxed initial structure. Spend some time to inspect the `AtomsTemplate` object and look at its attributes, properties and methods. This can be in the source code, but is probably easier in the variable inspector of a debugger that comes standard in most popular IDEs (VSCode, Pycharm)

In [3]:
template = AtomsTemplate(atoms)  # stores a copy of atoms
print('Inside AtomsTemplate:', [_ for _ in dir(template) if not _.startswith('_')])
visualise_template_blocks(template)

Inside AtomsTemplate: ['atoms', 'block_ids', 'block_labels', 'blocks', 'bonds', 'bonds_cg', 'cell', 'connections', 'graph', 'graph_cg', 'init', 'labels', 'labels_to_ids', 'make_supercell', 'merge_blocks', 'name', 'pbc', 'pos']


The original atoms object is broken down into 4 bricks, 24 linkers and 96 loose H atoms on all benzene rings. This decision is made in the `graph_from_atoms` function, called upon `AtomsTemplate` initialisation. It works out of the box for 'simple' MOFs like UiO-66, but is in no means general for framework materials. You can always implement a custom `graph_from_atoms` for your specific use case.

In [4]:
# initialise the AtomsTemplate using an external atomic graph
atoms_manual = atoms.copy()
graph_manual = graph_from_atoms(atoms_manual)
template_manual = AtomsTemplate(atoms_manual, graph=graph_manual)
visualise_template_blocks(template_manual)

# both templates are functionally equivalent, but not the same
assert template == template_manual

AssertionError: 

The default `graph_from_atoms` breaks down our framework into the 'smallest' chemically sound units. This behaviour is useful if we are interested in linker functionalisation (e.g., replacing a dangling -H with -NH<sub>2</sub>) but might not be desirable in general. You can merge building blocks by size to restore the original benzene ring.

In [5]:
# combine every isolated block with less than 2 atoms with its smallest neighbouring block
print('# of building blocks before:', len(template.blocks))
template.merge_blocks(min_size=2)
print('# of building blocks after:', len(template.blocks))
visualise_template_blocks(template)

# of building blocks before: 124
# of building blocks after: 28


Working in the conventional unit cell is instructive, but practical applications will need larger supercells to accomodate spatial disorder. You can scale up an `AtomsTemplate` using its `make_supercell` method.

In [6]:
template_super = template.make_supercell(2, 2, 2)
visualise_template_blocks(template_super)

Note that creating an `AtomsTemplate` is computationally costly for large structures. We need to generate a neighbourlist, analyse the topology and break down the system into building blocks. This scales poorly with system size. It is **much** more efficient to create the template from a small system and scale it up afterwards.

In [ ]:
%%timeit
# make a template from the unit cell and scale it up
template = AtomsTemplate(atoms)
template.make_supercell(4, 4, 4)

In [ ]:
%%timeit
# make a template from a supercell
atoms_super = atoms.repeat(4)
AtomsTemplate(atoms_super)

## The `ExtendedAtomsTemplate` class

To start modifying structures - create linker defects, swap metal ions etc. - we need the `ExtendedAtomsTemplate` class. Have a look at its additional methods; most of them should be quite self-explanatory.

In [7]:
template = ExtendedAtomsTemplate(atoms)
template.merge_blocks(2)
print('Inside ExtendedAtomsTemplate:', [_ for _ in dir(template) if not _.startswith('_')])

Inside ExtendedAtomsTemplate: ['atoms', 'block_ids', 'block_labels', 'blocks', 'bonds', 'bonds_cg', 'cell', 'connections', 'copy', 'dangling_bonds', 'get_atoms', 'ghost_atoms', 'graph', 'graph_cg', 'init', 'insert_block', 'labels', 'labels_to_ids', 'make_supercell', 'merge_blocks', 'name', 'pbc', 'pos', 'remove_blocks', 'rotate_blocks', 'swap_blocks', 'termination']


Let's start with a simple example: removing a single linker. The building blocks of a template are stored under the `blocks` attribute in order of descending size, meaning the first four are bricks. 

In [8]:
# blocks are BuildingBlock instances tied to a template
print(template.blocks[0], template.blocks[20], sep='\n')

# pass an iterable with the indices of blocks you want to remove
template.remove_blocks([20])
atoms_disordered = template.get_atoms(ghost_atoms=True, wrap=True)
ase.visualize.view(atoms_disordered)

BuildingBlock(H4C12O32Zr6, pos: [10.44, 10.44, 10.44], connections: 12)
BuildingBlock(H4C6, pos: [15.65, 5.22, 10.17], connections: 2)


<Popen: returncode: None args: ['/home/pieter/micromamba/envs/psiflow_dev/bi...>

Notice how the structure contains 2 dummy atoms with element 'X' where the linker was removed. These `ghost_atoms` represent unsaturated, dangling bonds that can be terminated to end up with a closed-shell neutral system. The default termination is simple -H caps, creating formate groups in UiO-66. Note that capping groups are simply placed at a reasonable bond distance. For further applications, it is a good idea to relax the atomic structure.

In [9]:
atoms_disordered = template.get_atoms(termination=True, wrap=True)
ase.visualize.view(atoms_disordered)

<Popen: returncode: None args: ['/home/pieter/micromamba/envs/psiflow_dev/bi...>

We can reinsert blocks into framework vacancies too. For this, we need a `StandaloneBuildingBlock` instance, which is not connected to any template. It is very easy to convert an existing `BuildingBlock`. Here, we start from a -NH<sub>2</sub> functionalised framework to contrast with the regular BDC linkers in UiO-66.

In [10]:
# create a StandaloneBuildingBlock from a BuildingBlock
file = root / 'data' / 'uio66nh2_unit.xyz'
atoms_nh2 = ase.io.read(file)
template_nh2 = AtomsTemplate(atoms_nh2)
template_nh2.merge_blocks(4)
block_bdc_nh2 = StandaloneBuildingBlock.from_block(template_nh2.blocks[4])
ase.visualize.view([atoms_nh2, block_bdc_nh2.get_atoms(ghost_atoms=True)])

<Popen: returncode: None args: ['/home/pieter/micromamba/envs/psiflow_dev/bi...>

/home/pieter/micromamba/envs/psiflow_dev/lib/python3.10/site-packages/ase/gui/images.py:99: UserWarning: Not all images have the same boundary conditions!
  warnings.warn('Not all images have the same boundary conditions!')


To fill the linker vacancy in our original template, we need to specify which dangling bonds should be reconnected by the new building block. The `insert_block` method will then try to fit the block inside our template topology.

In [11]:
# the template currently only has one linker vacancy, so 2 dangling bonds
dangling_bonds = template.dangling_bonds
print(dangling_bonds)

template.insert_block(block_bdc_nh2, dangling_bonds)
atoms_disordered = template.get_atoms(wrap=True)
ase.visualize.view(atoms_disordered)

{DanglingBond(labels=(248, 288), ids=(248, 288), pos=array([[13.60861488,  7.26443514, 10.31028601],
       [14.66272261,  6.20987392, 10.20640863]])), DanglingBond(labels=(245, 301), ids=(245, 301), pos=array([[17.69682524,  3.17523344, 10.31056884],
       [16.64245499,  4.22940445, 10.20633432]]))}


<Popen: returncode: None args: ['/home/pieter/micromamba/envs/psiflow_dev/bi...>

Manually selecting every `DanglingBond` pair (or e.g., triplet for tritopic linkers) can be tedious, especially for larger structures with many vacancies. You can replace blocks with the `swap_blocks` method, which combines a `remove_blocks` with several `insert_block` calls. Make sure that building block dimensions match approximately, or weird things will happen.

In [12]:
# replace multiple BDC linkers with the BDC-NH2 variant
# note that this operation changes the order of blocks in the template
template.swap_blocks([4, 8, 12], block_bdc_nh2)
atoms_disordered = template.get_atoms(wrap=True)
ase.visualize.view(atoms_disordered)

<Popen: returncode: None args: ['/home/pieter/micromamba/envs/psiflow_dev/bi...>

In [14]:
# the old an new blocks need to match topologically
# you cannot swap a 12-connected brick for a 2-connected linker
template.swap_blocks([0], block_bdc_nh2)

AssertionError: 

For asymmetric ditopic linkers, orientation can be an important degree of freedom. You can rotate blocks around their axis using the `rotate_blocks` nethod.

In [15]:
# rotate 3 linkers by 90 degrees
template.rotate_blocks([4, 5, 6], 90)
atoms_disordered = template.get_atoms(wrap=True)
ase.visualize.view(atoms_disordered)

<Popen: returncode: None args: ['/home/pieter/micromamba/envs/psiflow_dev/bi...>

This was a quick rundown of the main functionality provided by the `ExtendedAtomsTemplate` class. It has grown dynamically on an ad hoc basis, i.e., whenever needed. If your use case is not covered at this time, feel free to reach out and discuss on GitHub.

## The `AtomSelection` class

Modifying templates by block index quickly becomes cumbersome for large structures with thousands of building blocks. The `selection` module provides convenience functions to automatically select collections of building blocks. The main interface is accessed through an `AtomSelection` object. 

In [16]:
template = ExtendedAtomsTemplate(atoms)
template.merge_blocks(2)
template = template.make_supercell(3, 3, 1)
ase.visualize.view(template.get_atoms())

# create a selection for this particular template
selection = AtomSelection(template)
print(selection)
print('Inside AtomSelection:', [_ for _ in dir(selection) if not _.startswith('_')])

Selection(level=blocks, n_atoms=4104, n_blocks=252, selected=0)
Inside AtomSelection: ['active_ids', 'active_mask', 'atoms', 'blocks', 'copy', 'get_atoms_mask', 'is_consistent', 'level', 'map_atom_to_block', 'map_block_to_atoms', 'modify', 'n_atoms', 'n_blocks', 'n_selected', 'reset', 'select', 'toggle_level']


We control a selection by the `select` method that accepts `SelectionRule` instances. Below, we provide a non-exhaustive summation of possible combinations. Look at the source code for an overview of every `SelectionRule`. Implement custom rules by subclassing `SelectionRule`.

In [17]:
# select every block containing C
selection.select(SelectByElement('C'))
print(f'0\t| {selection}')
visualise_selection(selection)

# deselect every block containing Zr
selection.select(SelectByElement('Zr'), add=False)
print(f'1\t| {selection}')
visualise_selection(selection)

# reset to initial state
selection.reset()
print(f'2\t| {selection}')

# randomly select 20% of blocks
selection.select(SelectByChance(fraction=0.2))
print(f'3\t| {selection}')
visualise_selection(selection)

# select every block with 12 connections
selection.select(SelectByConnectivity(12))
print(f'4\t| {selection}')
visualise_selection(selection)

0	| Selection(level=blocks, n_atoms=4104, n_blocks=252, selected=252)
1	| Selection(level=blocks, n_atoms=4104, n_blocks=252, selected=216)
2	| Selection(level=blocks, n_atoms=4104, n_blocks=252, selected=0)
3	| Selection(level=blocks, n_atoms=4104, n_blocks=252, selected=50)
4	| Selection(level=blocks, n_atoms=4104, n_blocks=252, selected=79)


It's possible to combine multiple rules for additional flexibility. The `select` method will apply every rule to all blocks that are not yet actively selected (or every block that is already selected if `add=False`) and use the intersection of all rules.

In [18]:
# select every block containing C and not Zr
selection.reset()
selection.select(SelectByElement('C'), SelectByElement('Zr', invert=True))
visualise_selection(selection)

# select half of all bricks randomly
selection.reset()
selection.select(SelectByElement('Zr'), SelectByChance(fraction=0.5))
visualise_selection(selection)

Consecutive `select` calls enable more complex building block selections.

In [19]:
# select a random brick and then its surrounding linkers
selection.reset()
selection.select(SelectByElement('Zr'), SelectByChance(amount=1))
selection.select(SelectByAdjacency(1))
visualise_selection(selection)

# select brick 0 and then its closest bricks
selection.reset()
selection.select(SelectByIndex(0))
selection.select(SelectByAdjacency(2), SelectByAdjacency(1, invert=True))
visualise_selection(selection)

You can easily retrieve the indices of selected blocks to modify an `ExtendedAtomsTemplate` at scale.

In [20]:
# select a random brick and then its surrounding linkers
template_copy = template.copy()
selection = AtomSelection(template_copy)
selection.select(SelectByElement('Zr'), SelectByChance(amount=1))
selection.select(SelectByAdjacency(1))

# remove full selection from the template
ids = selection.active_ids
print('Selected indices:', ids)
template_copy.remove_blocks(ids)
atoms_disordered = template_copy.get_atoms(ghost_atoms=True, wrap=True)
ase.visualize.view(atoms_disordered)

# altering a template borks the selection instance
selection.select()

Selected indices: [ 24  48  57  87  96 114 129 150 165 204 210 237 249]


AssertionError: Selection is no longer consistent with template. Reinitialise..

An `AtomSelection` also has limited functionality at the atomic level. At present, this is very barebones.

In [ ]:
# select 20% of Zr atoms
selection = AtomSelection(template_copy)
print(selection)
selection.toggle_level()
selection.select(SelectByElement('Zr'), SelectByChance(fraction=0.2))
print(selection)
visualise_selection(selection)

# swap out for Hf
template_copy.atoms.numbers[selection.active_ids] = 72
atoms_disordered = template_copy.get_atoms(wrap=True)
ase.visualize.view(atoms_disordered)

## Playground

You reached the end of this short tutorial. Have fun messing about.